In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
df = pd.read_csv("NY property data.csv")
df.head()

,RECORD,BBLE,B,BLOCK,LOT,EASEMENT,OWNER,BLDGCL,TAXCLASS,LTFRONT,...,BLDFRONT,BLDDEPTH,AVLAND2,AVTOT2,EXLAND2,EXTOT2,EXCD2,PERIOD,YEAR,VALTYPE
0,1,1000010101,1,1,101,NaN,U S GOVT LAND & BLDGS,P7,4,500,...,0,0,3775500.0,8613000.0,3775500.0,8613000.0,NaN,FINAL,2010/11,AC-TR
1,2,1000010201,1,1,201,NaN,U S GOVT LAND & BLDGS,Z9,4,27,...,0,0,11111400.0,80690400.0,11111400.0,80690400.0,NaN,FINAL,2010/11,AC-TR
2,3,1000020001,1,2,1,NaN,DEPT OF GENERAL SERVI,Y7,4,709,...,709,564,32321790.0,40179510.0,32321790.0,40179510.0,NaN,FINAL,2010/11,AC-TR
3,4,1000020023,1,2,23,NaN,DEPARTMENT OF BUSINES,T2,4,793,...,85,551,13644000.0,15750000.0,13644000.0,15750000.0,NaN,FINAL,2010/11,AC-TR
4,5,1000030001,1,3,1,NaN,PARKS AND RECREATION,Q1,4,323,...,89,57,106348680.0,107758350.0,106348680.0,107758350.0,NaN,FINAL,2010/11,AC-TR


#### 1. remove not interested owners

In [3]:
remove_list = ['PARKCHESTER PRESERVAT', 'PARKS AND RECREATION', 'DCAS', 'HOUSING PRESERVATION', 
        'CITY OF NEW YORK', 'DEPT OF ENVIRONMENTAL', 'BOARD OF EDUCATION', 'NEW YORK CITY HOUSING', 
        'CNY/NYCTA', 'NYC HOUSING PARTNERSH', 'DEPARTMENT OF BUSINES', 'DEPT OF TRANSPORTATIO', 
        'MTA/LIRR', 'PARCKHESTER PRESERVAT', 'MH RESIDENTIAL 1, LLC', 'LINCOLN PLAZA ASSOCIA', 
        'UNITED STATES OF AMER', 'U S GOVERNMENT OWNRD', 'THE CITY OF NEW YORK', 'NYS URBAN DEVELOPMENT',
        'NYS DEPT OF ENVIRONME', 'CULTURAL AFFAIRS', 'DEPT OF GENERAL SERVI', 'DEPT RE-CITY OF NY']
property_data = df[~df.OWNER.isin(remove_list)].copy()

#### 2. fill in `ZIP`

In [4]:
property_data = property_data.reset_index()

In [5]:
missing_zips = np.where(pd.isnull(property_data['ZIP']))[0]
len(missing_zips)

21772

In [6]:
for i in range(len(missing_zips)):
    if(property_data.loc[missing_zips[i]+1, 'ZIP'] == property_data.loc[missing_zips[i]-1, 'ZIP']):
        property_data.loc[missing_zips[i], 'ZIP'] = property_data.loc[missing_zips[i]-1, 'ZIP']

In [7]:
missing_zips = np.where(pd.isnull(property_data['ZIP']))[0]
len(missing_zips)

10245

In [8]:
for i in range(len(missing_zips)):
    property_data.loc[missing_zips[i], 'ZIP'] = property_data.loc[missing_zips[i]-1, 'ZIP']
        

In [9]:
missing_zips = np.where(pd.isnull(property_data['ZIP']))[0]
len(missing_zips)

0

#### 3. fill in `FULLVAL`, `AVLAND`, and `AVTOT`

In [10]:
property_data['FULLVAL'].replace('NaN', 0)
temp = property_data[property_data['FULLVAL']!=0]
mean_fullval = temp.groupby('TAXCLASS')['FULLVAL'].mean()
mean_fullval

TAXCLASS
1     5.700059e+05
1A    3.352942e+05
1B    3.738399e+05
1C    7.615359e+05
1D    2.233614e+07
2     8.005832e+05
2A    8.639066e+05
2B    1.254525e+06
2C    7.723493e+05
3     8.510005e+04
4     2.772747e+06
Name: FULLVAL, dtype: float64

In [11]:
property_data['AVLAND'].replace('NaN', 0)
temp = property_data[property_data['AVLAND']!=0]
mean_avland = temp.groupby('TAXCLASS')['AVLAND'].mean()
mean_avland

TAXCLASS
1      14901.860425
1A      2247.472465
1B      9185.394750
1C      8225.658898
1D    709303.793103
2      88471.102578
2A     31370.740197
2B     54607.394911
2C     25635.944099
3      28135.636364
4     444996.049088
Name: AVLAND, dtype: float64

In [12]:
property_data['AVTOT'].replace('NaN', 0)
temp = property_data[property_data['AVTOT']!=0]
mean_avtot = temp.groupby('TAXCLASS')['AVTOT'].mean()
mean_avtot

TAXCLASS
1     2.502549e+04
1A    1.437036e+04
1B    9.189080e+03
1C    2.898447e+04
1D    1.166866e+06
2     3.602638e+05
2A    7.963525e+04
2B    1.790961e+05
2C    1.170418e+05
3     3.829505e+04
4     1.294237e+06
Name: AVTOT, dtype: float64

In [13]:
for index in mean_fullval.index:
    property_data.loc[(property_data['FULLVAL'] == 0) & (property_data['TAXCLASS'] == index),'FULLVAL'] = mean_fullval[index]
    property_data.loc[(property_data['AVLAND'] == 0) & (property_data['TAXCLASS'] == index),'AVLAND'] = mean_avland[index]
    property_data.loc[(property_data['AVTOT'] == 0) & (property_data['TAXCLASS'] == index),'AVTOT'] = mean_avtot[index]

#### 4. fill in `STORIES`

In [14]:
temp = property_data[property_data['STORIES'].isnull()]
len(temp)

43968

In [15]:
len(property_data[property_data['STORIES']==0])

0

In [16]:
temp['TAXCLASS'].value_counts()

1B    19055
4     17979
2      3434
3      2416
1       877
2C      138
2B       34
2A       30
1A        5
Name: TAXCLASS, dtype: int64

In [17]:
mean_stories = property_data.groupby('TAXCLASS')['STORIES'].mean()
mean_stories

TAXCLASS
1      2.111633
1A     1.656867
1B     4.000000
1C     3.052748
1D     1.068966
2     16.310549
2A     2.844032
2B     4.001635
2C     4.744977
3      1.000000
4      5.517257
Name: STORIES, dtype: float64

In [18]:
property_data['STORIES'] = property_data['STORIES'].fillna(value=0)
for index in mean_stories.index:
    property_data.loc[(property_data['STORIES'] == 0) & (property_data['TAXCLASS'] == index), 'STORIES'] = mean_stories[index]

In [19]:
#property_data.head().transpose()

#### 5. fill in `LTFRONT`, `LTDEPTH`, `BLDFRONT`, and `BLDDEPTH`

In [20]:
#convert 0 & 1 to NA to prepare for mean calculation
property_data.loc[property_data['LTFRONT']==0,'LTFRONT']=np.nan
property_data.loc[property_data['LTDEPTH']==0,'LTDEPTH']=np.nan
property_data.loc[property_data['BLDFRONT']==0,'BLDFRONT']=np.nan
property_data.loc[property_data['BLDDEPTH']==0,'BLDDEPTH']=np.nan

property_data.loc[property_data['LTFRONT']==1,'LTFRONT']=np.nan
property_data.loc[property_data['LTDEPTH']==1,'LTDEPTH']=np.nan
property_data.loc[property_data['BLDFRONT']==1,'BLDFRONT']=np.nan
property_data.loc[property_data['BLDDEPTH']==1,'BLDDEPTH']=np.nan


In [21]:
#mean
mean_LTFRONT=property_data.groupby(property_data['TAXCLASS'])['LTFRONT'].mean()
mean_LTDEPTH=property_data.groupby(property_data['TAXCLASS'])['LTDEPTH'].mean()
mean_BLDFRONT=property_data.groupby(property_data['TAXCLASS'])['BLDFRONT'].mean()
mean_BLDDEPTH=property_data.groupby(property_data['TAXCLASS'])['BLDDEPTH'].mean()

In [22]:
#imputation
for index in mean_LTFRONT.index:
    property_data.loc[(property_data['LTFRONT'].isnull())&(property_data['TAXCLASS']==index),'LTFRONT']=mean_LTFRONT[index]    
    property_data.loc[(property_data['LTDEPTH'].isnull())&(property_data['TAXCLASS']==index),'LTDEPTH']=mean_LTDEPTH[index]
    property_data.loc[(property_data['BLDFRONT'].isnull())&(property_data['TAXCLASS']==index),'BLDFRONT']=mean_BLDFRONT[index]
    property_data.loc[(property_data['BLDDEPTH'].isnull())&(property_data['TAXCLASS']==index),'BLDDEPTH']=mean_BLDDEPTH[index]


#### 6. make variables

In [23]:
# zip to int
property_data['ZIP'] = property_data['ZIP'].astype(str)
property_data['ZIP3'] = property_data['ZIP'].str[:3]

In [24]:
#create 3 variables
property_data['ltsize'] = property_data.LTFRONT * property_data.LTDEPTH
property_data['bldsize'] = property_data.BLDFRONT * property_data.BLDDEPTH
property_data['bldvol'] = property_data.bldsize * property_data.STORIES

In [25]:
#create 9 variables
property_data['r1'] = property_data['FULLVAL'] / property_data['ltsize']
property_data['r2'] = property_data['FULLVAL'] / property_data['bldsize']
property_data['r3'] = property_data['FULLVAL'] / property_data['bldvol']

property_data['r4'] = property_data['AVLAND'] / property_data['ltsize']
property_data['r5'] = property_data['AVLAND'] / property_data['bldsize']
property_data['r6'] = property_data['AVLAND'] / property_data['bldvol']

property_data['r7'] = property_data['AVTOT'] / property_data['ltsize']
property_data['r8'] = property_data['AVTOT'] / property_data['bldsize']
property_data['r9'] = property_data['AVTOT'] / property_data['bldvol']

In [26]:
#create 45 variables
ninevars = ['r1', 'r2', 'r3', 'r4', 'r5', 'r6', 'r7', 'r8', 'r9']

zip5_mean = property_data.groupby('ZIP')[ninevars].mean()
zip3_mean = property_data.groupby('ZIP3')[ninevars].mean()
taxclass_mean = property_data.groupby('TAXCLASS')[ninevars].mean()
borough_mean = property_data.groupby('B')[ninevars].mean()

property_data = property_data.join(zip5_mean, on='ZIP', rsuffix = '_zip5')
property_data = property_data.join(zip3_mean, on='ZIP3', rsuffix = '_zip3')
property_data = property_data.join(taxclass_mean, on='TAXCLASS', rsuffix = '_taxclass')
property_data = property_data.join(borough_mean, on='B', rsuffix = '_boro')
rsuffix = ['_zip5', '_zip3', '_taxclass', '_boro']

for var in ninevars:
    for r in rsuffix:
        property_data[str(var)+r] = property_data[var] / property_data[str(var)+r]

In [27]:
nyvar = property_data.iloc[:, -45:]
submit = pd.DataFrame(nyvar.count(), columns = ['count'])
submit['mean'] = round(nyvar.mean(), 2)
submit['std'] = round(nyvar.std(), 2)
submit['min'] = nyvar.min()
submit['25%'] = round(nyvar.quantile(0.25), 2)
submit['50%'] = round(nyvar.quantile(0.5), 2)
submit['75%'] = round(nyvar.quantile(0.75), 2)
submit['max'] = round(nyvar.max(), 2)
submit

,count,mean,std,min,25%,50%,75%,max
r1,1046826,213.57,532.56,3.905765e-04,76.26,152.86,243.18,138637.34
r2,1046826,536.42,1027.44,7.466192e-04,211.82,500.00,683.82,310551.57
r3,1046826,244.08,498.82,8.983394e-05,82.50,239.06,336.23,227500.00
r4,1046826,10.46,69.98,8.333333e-06,2.35,4.60,7.23,22249.80
r5,1046826,22.82,378.51,2.096125e-04,6.20,14.98,20.29,334636.37
r6,1046826,10.04,178.33,1.038033e-05,2.02,7.15,10.04,111545.46
r7,1046826,26.83,203.87,8.333333e-06,5.28,8.58,13.53,64711.85
r8,1046826,50.46,932.20,3.062474e-04,17.47,26.98,36.00,871362.22
r9,1046826,19.07,327.02,6.049251e-05,7.09,12.68,17.58,290454.07
r1_zip5,1046826,1.00,2.16,2.228349e-06,0.46,0.92,1.23,747.76


In [28]:
submit.to_csv('variable_statistics.csv')